In [68]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

database = os.environ.get('NEO4J_DATABASE')
graph = Neo4jGraph(database=database)

In [69]:
from graphdatascience import GraphDataScience 

gds = GraphDataScience( 
    os.environ[ "NEO4J_URI" ], 
    auth=(os.environ[ "NEO4J_USERNAME" ], os.environ[ "NEO4J_PASSWORD" ]) 
)

In [70]:
gds.graph.drop("communities")
G, result = gds.graph.project(
    "communities",  #  Graph name
    "__Entity__",  #  Node projection
    {
        "_ALL_": {
            "type": "*",
            "orientation": "UNDIRECTED",
            "properties": {"weight": {"property": "*", "aggregation": "COUNT"}},
        }
    },
)

In [71]:
wcc = gds.wcc.stats(G)
print(f"Component count: {wcc['componentCount']}")
print(f"Component distribution: {wcc['componentDistribution']}")

Component count: 274
Component distribution: {'min': 1, 'p5': 1, 'max': 185, 'p999': 185, 'p99': 14, 'p1': 1, 'p10': 1, 'p90': 1, 'p50': 1, 'p25': 1, 'p75': 1, 'p95': 5, 'mean': 2.0437956204379564}


In [72]:
gds.leiden.write(
    G,
    writeProperty="communities",
    includeIntermediateCommunities=True,
    relationshipWeightProperty="weight",
)

writeMillis                                                             22
nodePropertiesWritten                                                  560
ranLevels                                                                3
didConverge                                                           True
nodeCount                                                              560
communityCount                                                         281
communityDistribution    {'min': 1, 'p5': 1, 'max': 54, 'p999': 54, 'p9...
modularity                                                        0.722143
modularities             [0.647866700528563, 0.7200064284693894, 0.7221...
postProcessingMillis                                                    39
preProcessingMillis                                                      0
computeMillis                                                           82
configuration            {'writeProperty': 'communities', 'theta': 0.01...
Name: 0, dtype: object

In [73]:
graph.query("""
MATCH (e:`__Entity__`)
UNWIND range(0, size(e.communities) - 1 , 1) AS index
CALL {
  WITH e, index
  WITH e, index
  WHERE index = 0
  MERGE (c:`__Community__` {id: toString(index) + '-' + toString(e.communities[index])})
  ON CREATE SET c.level = index
  MERGE (e)-[:IN_COMMUNITY]->(c)
  RETURN count(*) AS count_0
}
CALL {
  WITH e, index
  WITH e, index
  WHERE index > 0
  MERGE (current:`__Community__` {id: toString(index) + '-' + toString(e.communities[index])})
  ON CREATE SET current.level = index
  MERGE (previous:`__Community__` {id: toString(index - 1) + '-' + toString(e.communities[index - 1])})
  ON CREATE SET previous.level = index - 1
  MERGE (previous)-[:IN_COMMUNITY]->(current)
  RETURN count(*) AS count_1
}
RETURN count(*)
""")

[{'count(*)': 1680}]

In [74]:
graph.query("""
MATCH (c:__Community__)<-[:IN_COMMUNITY*]-(:__Entity__)<-[:HAS_ENTITY]-(d:__Chunk__)
WITH c, count(distinct d) AS rank
SET c.rank = rank;
""")

[]

In [75]:
import pandas as pd
import numpy as np

community_size = graph.query(
    """
MATCH (c:__Community__)<-[:IN_COMMUNITY*]-(e:__Entity__)
WITH c, count(distinct e) AS entities
RETURN split(c.id, '-')[0] AS level, entities
"""
)
community_size_df = pd.DataFrame.from_records(community_size)
percentiles_data = []
for level in community_size_df["level"].unique():
    subset = community_size_df[community_size_df["level"] == level]["entities"]
    num_communities = len(subset)
    percentiles = np.percentile(subset, [25, 50, 75, 90, 99])
    percentiles_data.append(
        [
            level,
            num_communities,
            percentiles[0],
            percentiles[1],
            percentiles[2],
            percentiles[3],
            percentiles[4],
            max(subset)
        ]
    )

# Create a DataFrame with the percentiles
percentiles_df = pd.DataFrame(
    percentiles_data,
    columns=[
        "Level",
        "Number of communities",
        "25th Percentile",
        "50th Percentile",
        "75th Percentile",
        "90th Percentile",
        "99th Percentile",
        "Max"
    ],
)
percentiles_df

,Level,Number of communities,25th Percentile,50th Percentile,75th Percentile,90th Percentile,99th Percentile,Max
0,0,309,1.0,1.0,1.0,3.0,11.92,38
1,1,283,1.0,1.0,1.0,1.0,21.00,51
2,2,281,1.0,1.0,1.0,1.0,21.00,54


In [76]:
community_info = graph.query("""
MATCH (c:`__Community__`)<-[:IN_COMMUNITY*]-(e:__Entity__)
WITH c, collect(e ) AS nodes
WHERE size(nodes) > 1
CALL apoc.path.subgraphAll(nodes[0], {
 whitelistNodes:nodes
})
YIELD relationships
RETURN c.id AS communityId, 
       [n in nodes | apoc.map.removeKeys(n{.*, type: [el in labels(n) WHERE el <> '__Entity__'][0]}, ['embedding', 'wcc', 'communities'])] AS nodes,
       [r in relationships | {start: startNode(r).id, type: type(r), end: endNode(r).id, description: r.description}] AS rels
""")
community_info[0]

{'communityId': '0-56',
 'nodes': [{'id': '被保險人',
   'description': '主契約之被保險人或其配偶',
   'type': 'Concept',
   'sources': ['台灣人壽龍實在住院醫療健康保險附約.pdf', '台灣人壽新住院醫療保險附約.pdf']},
  {'id': '疾病',
   'description': '被保險人自本附約生效日（或復效日）起所發生之疾病',
   'type': 'Concept',
   'sources': ['台灣人壽龍實在住院醫療健康保險附約.pdf', '台灣人壽新住院醫療保險附約.pdf']},
  {'id': '傷害',
   'description': '被保險人於本附約有效期間內，遭受意外傷害事故，因而蒙受之傷害',
   'type': 'Concept',
   'sources': ['台灣人壽龍實在住院醫療健康保險附約.pdf', '台灣人壽新住院醫療保險附約.pdf']},
  {'id': '醫院',
   'description': '依照醫療法規定領有開業執照並設有病房收治病人之公、私立及醫療法人醫院',
   'type': 'Concept',
   'sources': ['台灣人壽龍實在住院醫療健康保險附約.pdf', '台灣人壽新住院醫療保險附約.pdf']},
  {'id': '診所',
   'description': '依照醫療法規定領有開業執照之診所',
   'type': 'Concept',
   'sources': ['台灣人壽龍實在住院醫療健康保險附約.pdf', '台灣人壽新住院醫療保險附約.pdf']},
  {'id': '住院',
   'description': '被保險人經醫師診斷其疾病或傷害必須入住醫院，且正式辦理住院手續並確實在醫院接受診療者。但不包含全民健康保險法第五十一條所稱之日間住院及精神衛生法第三十五條所稱之日間留院',
   'type': 'Concept',
   'sources': ['台灣人壽龍實在住院醫療健康保險附約.pdf', '台灣人壽新住院醫療保險附約.pdf']},
  {'id': '特定處置費用保險金',
   'descrip

In [77]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
# from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0
)

In [78]:
# from langchain_experimental.llms.ollama_functions import OllamaFunctions
# model_name = 'qwen2:72b-instruct-q8_0'
# llm = OllamaFunctions(model=model_name, temperature=0)
# llm

In [79]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

community_template = """
{community_info}
Summary:"""  # noqa: E501

community_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """請根據提供同社區的資訊包含 nodes 與 relationships, 產生同社區的自然語言的摘要資訊, 
當提供多個來源(sources)資訊時，需將共同的資訊和各來源獨立提供的資訊分別呈現；對於共同資訊部分，統一匯集與展示；對於各個來源的獨立資訊，則按來源分開顯示。此外，如果僅有單一來源資訊，則只需呈現來源檔案名稱即可. No pre-amble.""",
        ),
        (
            "system", "使用繁體中文回應"
        ),
        ("human", community_template),
    ]
)

community_chain = community_prompt | llm | StrOutputParser()

In [80]:

def prepare_string(data):
    nodes_str = "Nodes are:\n"
    for node in data['nodes']:
        desc_list = []
        for key in node:
            if node[key] is not None:
                desc_list.append(f"{key}: {node[key]}")
        nodes_str += ', '.join(desc_list)

    rels_str = "Relationships are:\n"
    for rel in data['rels']:
        start = rel['start']
        end = rel['end']
        rel_type = rel['type']
        if 'description' in rel and rel['description']:
            description = f", description: {rel['description']}"
        else:
            description = ""
        rels_str += f"({start})-[:{rel_type}]->({end}){description}\n"

    return nodes_str + "\n" + rels_str

def process_community(community):
    stringify_info = prepare_string(community)
    summary = community_chain.invoke({'community_info': stringify_info})
    return {"community": community['communityId'], "summary": summary}

process_community(community_info[0])

{'community': '0-56',
 'summary': '共同資訊:\n- 被保險人: 主契約之被保險人或其配偶\n- 疾病: 被保險人自本附約生效日（或復效日）起所發生之疾病\n- 傷害: 被保險人於本附約有效期間內，遭受意外傷害事故，因而蒙受之傷害\n- 醫院: 依照醫療法規定領有開業執照並設有病房收治病人之公、私立及醫療法人醫院\n- 診所: 依照醫療法規定領有開業執照之診所\n- 住院: 被保險人經醫師診斷其疾病或傷害必須入住醫院，且正式辦理住院手續並確實在醫院接受診療者。但不包含全民健康保險法第五十一條所稱之日間住院及精神衛生法第三十五條所稱之日間留院\n- 保險範圍: 被保險人於本附約有效期間內因第二條約定之疾病或傷害住院診療或接受門診外科手術治療時，本公司按其投保計劃內容，依照本附約的約定給付保險金\n- 附表二: 手術名稱及費用表\n- 住院診療\n- 保單年度: 同一保單年度之「門診手術費用保險金」最高給付次數以6次為限\n- 全民健康保險: 保險對象身分住院或接受門診外科手術診療\n- 門診治療\n- 出院\n- 第十五條: 被保險人因特定原因住院或門診診療者，本公司不負給付保險金的責任\n- 故意行為: 包括自殺及自殺未遂\n- 犯罪行為: 被保險人之犯罪行為\n- 非法施用毒品: 非法施用防制毒品相關法令所稱之毒品\n- 美容手術: 美容手術、外科整型\n- 天生畸形: 外觀可見之天生畸形\n- 牙科手術: 非因當次住院事故治療之目的所進行之牙科手術\n- 裝設義齒: 裝設義齒、義肢、義眼、眼鏡、助聽器或其它附屬品\n- 門診外科手術治療\n\n台灣人壽龍實在住院醫療健康保險附約.pdf:\n- 特定處置費用保險金: 實支實付之給付\n- 住院前後門診保險金: 被保險人於本附約有效期間內因第五條之約定而住院診療或接受門診手術者，其於入院診療或接受門診手術前七日內；與出院後或接受門診手術後十四日內，因同一事故門診治療時，本公司每次依其投保計劃給付「住院前後門診保險金」。同一日之「住院前後門診保險金」最高給付次數以1次為限\n- 第二條\n- 特定處置\n- 門診手術或特定處置費用保險金限額\n- 第五條\n\n台灣人壽新住院醫療保險附約.pdf:\n- 特別病房: 加護病房或燒燙傷病房\n- 門診外科手術: 被保險人於門診時所接受之

In [81]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time

def process_community_with_llm(community_info, max_retry=0):
    '''
    params:
        community_info: [ { 
                            'communityId': str, 'nodes': [{'id': str, 'description': str|None, 'type': str}, ...], 
                            'rels': [{'start': str, 'description': str|None, 'type': str, 'end': 'str}, ...]
                          },
                          ... ]
        max_retry: 最多嘗試次數, 假設為2, 則最多遞迴執行 2+1=3次
    '''
    summaries = []
    faild_communities = []
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(process_community, community): community for community in community_info}

        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing communities"):
            try:
                summaries.append(future.result())
            except Exception as e:
                community = futures[future]
                print(f'process community faild!:{community}, error:\n{e}')
                faild_communities.append(community)
    if len(faild_communities) > 0 and max_retry > 0:
        time.sleep(30) # 防止超出限制的情境, 等待一段時間後再嘗試
        summaries.extend(process_community_with_llm(faild_communities, max_retry=max_retry-1))
    return summaries
summaries = process_community_with_llm(community_info, max_retry=3)

Processing communities: 100%|██████████| 108/108 [01:32<00:00,  1.16it/s]


In [82]:
summaries

[{'community': '0-204',
  'summary': '共同資訊:\n- 醫師: 領有醫師證書與執業執照，合法執業者 (Concept)\n\n來源: 台灣人壽龍實在住院醫療健康保險附約.pdf\n- 醫師: 領有醫師證書與執業執照，合法執業者 (Concept)\n\n來源: 台灣人壽新住院醫療保險附約.pdf\n- 醫師: 領有醫師證書與執業執照，合法執業者 (Concept)\n- 危及生命之情形 (情形)\n\n關係:\n- 醫師 診斷 危及生命之情形'},
 {'community': '0-123',
  'summary': '共同資訊:\n- 意外傷害事故: 非由疾病引起之外來突發事故 (Concept)\n\n台灣人壽龍實在住院醫療健康保險附約.pdf:\n- 裝設義肢 (行為)\n- 裝設義眼 (行為)\n- 裝設眼鏡 (行為)\n- 裝設助聽器 (行為)\n- 裝設附屬品 (行為)\n- 關係:\n  - 裝設助聽器 例外於 意外傷害事故\n  - 裝設義眼 例外於 意外傷害事故\n  - 裝設附屬品 例外於 意外傷害事故\n  - 裝設眼鏡 例外於 意外傷害事故\n  - 裝設義肢 例外於 意外傷害事故\n\n台灣人壽新住院醫療保險附約.pdf:\n- 無其他獨立資訊'},
 {'community': '1-234',
  'summary': '共同資訊:\n- 醫師: 領有醫師證書與執業執照，合法執業者\n- 腫瘤: 腫瘤相關手術\n- 危及生命之情形\n- 放射線治療\n- 化學治療\n- 惡性瘤之外科切除: 20%\n- 腔鞘囊腫: 2%\n- 粘腋膜、皮膚和皮下組織之惡性瘤: 10%\n- 其他一個或多個良性瘤: 5%\n\n關係:\n- 醫師診斷腫瘤\n- 醫師診斷危及生命之情形\n- 腫瘤包含粘腋膜、皮膚和皮下組織之惡性瘤\n- 腫瘤包含惡性瘤之外科切除\n- 腫瘤包含腔鞘囊腫\n- 腫瘤包含其他一個或多個良性瘤\n- 腫瘤治療放射線治療\n- 腫瘤治療化學治療\n\n來源: \n- 台灣人壽龍實在住院醫療健康保險附約.pdf\n- 台灣人壽新住院醫療保險附約.pdf'},
 {'community': '2-100',
  'summary': '共同資訊:\n- 醫師: 領有醫師證書與執業執照，合法執業

In [83]:
graph.query("""
UNWIND $data AS row
MERGE (c:__Community__ {id:row.community})
SET c.summary = row.summary
""", params={"data": summaries})

[]

In [84]:
graph.query('''
MATCH (n:__Entity__)
WHERE n.description IS NOT NULL and apoc.meta.cypher.type(n.description) <> 'STRING'
REMOVE n.embedding
SET n.description = apoc.text.join(n.description, "\n---\n")
RETURN n.description
''')

[{'n.description': '90,000\n---\n120,000'},
 {'n.description': '600\n---\n900'},
 {'n.description': '並附心臟科專科醫師診斷證明或心電圖檢查認定須剖腹產者\n---\n嚴重心律不整，並附心臟科專科醫師診斷證明或心電圖檢查認定須剖腹產者'},
 {'n.description': '1,000\n---\n1,500'},
 {'n.description': '160,000\n---\n180,000'},
 {'n.description': '40,000\n---\n60,000'},
 {'n.description': '2,000\n---\n3,000'},
 {'n.description': '50%\n---\n32%'},
 {'n.description': '每次\n---\n7%'},
 {'n.description': '每次\n---\n5%'}]

In [85]:
graph.query(
    """
MATCH (n:`__Community__`)<-[:IN_COMMUNITY]-()<-[:HAS_ENTITY]-(c)
WITH n, count(distinct c) AS chunkCount
SET n.weight = chunkCount"""
)

[]